In [ ]:
import librosa
import importlib
import src.preprocess
import pandas as pd
import tensorflow_io as tfio
import tensorflow as tf
import numpy as np
importlib.reload(src.preprocess)

GS_DATA = "gs://bird-clef-kimmo/data"
TRAIN_SHORT_AUDIO_DATA = f"{GS_DATA}/train_short_audio"

In [ ]:
!gsutil ls $GS_DATA

In [ ]:
TRAIN_METADATA_CSV = f"{GS_DATA}/train_metadata.csv"

train_metadata_ds = tf.data.experimental.make_csv_dataset(
    TRAIN_METADATA_CSV, batch_size=1)

In [ ]:
def read_file(url):
    print(f"Reading file: {url}")
    return tf.squeeze(tfio.audio.AudioIOTensor(url).to_tensor())

def process_row(row):
    filename = row["filename"]
    primary_label = row["primary_label"]
    file_url = TRAIN_SHORT_AUDIO_DATA + "/" + primary_label + "/" + filename # tf.py_function(lambda filenames: [f"{GS_DATA}/{filename}" for filename in filenames], [filename], [tf.string])
    wave_form = tf.py_function(read_file, [tf.squeeze(file_url)], [tf.float32])
    return {**row, "file_url": file_url, "wave_form": wave_form}

for row in train_metadata_ds.map(process_row).take(4):
    print(row)


In [ ]:
a = tf.constant(np.array([["a"], ["c"]]))
b = tf.map_fn(lambda x: tf.expand_dims(x, axis=0), a)
print(b)
